## Simple source modeling
Trying to iron out all the confusion on how to calculate source parameters

In [ ]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
import pandas as pd

from obspy import Trace, Stream, UTCDateTime
import obsplus
from obsplus.utils.time import to_utc

from mopy import StatsGroup, TraceGroup, SpectrumGroup

t, a, b, c, f = sym.symbols('t a b c f')

## Source Model

We will use a simple guassian as a source model.

In [ ]:
# Get gausian and derivative/integral
gaus = a * sym.exp((-(t - b)**2) / (2 * c **2))
d_gaus = sym.diff(gaus, t)
int_gaus = sym.integrate(gaus, t)

In [ ]:
# get freq domain
F_gaus = sym.fourier_transform(gaus, t, f).rewrite(sym.Integral)
F_d_gaus = sym.fourier_transform(d_gaus, t, f).rewrite(sym.Integral)
F_int_gaus = sym.fourier_transform(int_gaus, t, f).rewrite(sym.Integral)

In [ ]:
gaus

In [ ]:
d_gaus

## Get numpy functions

In [ ]:
get_gaus = sym.lambdify([t, a, b, c], gaus)
get_d_gaus = sym.lambdify([t, a, b, c], d_gaus)
get_int_gaus = sym.lambdify([t, a, b, c], int_gaus)

## Next generate source time series and plot

In [ ]:
t1, t2, dt = 0, 10.24, 0.01
a, b, c = 0.1, 5, np.sqrt(3)
x_orig = np.arange(t1, t2+1, dt)
x = x_orig
# get source in 1) displacement 2) velocity
# dt = 0.01
# a, b, c = 0.1, 5, np.sqrt(2)
# # x = np.arange(0, 10/2, dt)
# x = np.arange(0, 10, dt)
ns = {}

# Without zero-padding
# source_disp_st = 
source_disp = np.zeros(len(x))
source_disp[0:len(x)] = get_gaus(x, a=a, b=b, c=c)  # displacement
ns["source"] = len(x)
source_vel = np.zeros(len(x))
source_vel[0:len(x)] = get_d_gaus(x, a=a, b=b, c=c)  # velocity
# With zero padding and a serious discontinuity
# x = np.arange(t1, (t2+1)/2, dt)
# dis_source_disp = np.zeros(len(x)*2)
# dis_source_disp[0:len(x)] = get_gaus(x, a=a, b=b, c=c)  # displacement
# dis_source_vel = np.zeros(len(x)*2)
# dis_source_vel[0:len(x)] = get_d_gaus(x, a=a, b=b, c=c)  # velocity
x = np.arange(t1, (t2+1)*0.75, dt)
ns["discosninuity"] = len(x)
dis_source_disp = np.zeros(len(x_orig))
dis_source_disp[0:len(x)] = get_gaus(x, a=a, b=b, c=c)  # displacement
dis_source_vel = np.zeros(len(x_orig))
dis_source_vel[0:len(x)] = get_d_gaus(x, a=a, b=b, c=c)  # velocity
# With zero padding, but relatively continuous
# x = np.arange(t1, (t2+1)/2, dt)
# zp_source_disp = np.zeros(len(x)*2)
# zp_source_disp[0:len(x)] = get_gaus(x, a=a, b=b/2, c=c/2)  # displacement
# zp_source_vel = np.zeros(len(x)*2)
# zp_source_vel[0:len(x)] = get_d_gaus(x, a=a, b=b/2, c=c/2)  # velocity
x = np.arange(t1, (t2+1)*0.75, dt)
ns["zero_padded"] = len(x)
zp_source_disp = np.zeros(len(x_orig))
zp_source_disp[0:len(x)] = get_gaus(x, a=a, b=b*0.75, c=c*0.75)  # displacement
zp_source_vel = np.zeros(len(x_orig))
zp_source_vel[0:len(x)] = get_d_gaus(x, a=a, b=b*0.75, c=c*0.75)  # velocity

x = x_orig

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))
ax1.plot(x, source_disp)
ax2.plot(x, dis_source_disp)
ax3.plot(x, zp_source_disp)
# plt.plot(x, source_disp)
ax1.set_ylabel('displacement amplitude (m)')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))
ax1.plot(x, source_vel)
ax2.plot(x, dis_source_vel)
ax3.plot(x, zp_source_vel)
ax1.set_ylabel('velocity amplitude (m/s)')

### Put all three source time series into MoPy

In [ ]:
ds = obsplus.load_dataset("coal_node")
# Load up some events
cat = ds.event_client.get_events()
# Load up a station inventory
inv = ds.station_client.get_stations()
# Generate a StatsGroup that can be used to create another StatsGroup
sg = StatsGroup(catalog=cat, inventory=inv)

# Two streams, one has the full data, the other has only part of it
# (to see the effects of zero padding the data)
df_contents = {
    "event_id": ["event1", "event1", "event1"],
    "seed_id": ["UK.STA1..HHZ", "UK.STA2..HHZ", "UK.STA3..HHZ"],
    "seed_id_less": ["UK.STA1..HH", "UK.STA2..HH", "UK.STA3..HH"],
    "phase_hint": ["P", "P", "P"],
    "time": [np.datetime64("2020-01-01T00:00:01"), np.datetime64("2020-01-01T00:00:01"), np.datetime64("2020-01-01T00:00:01")],
    "starttime": [np.datetime64("2020-01-01T00:00:00"), np.datetime64("2020-01-01T00:00:00"), np.datetime64("2020-01-01T00:00:00"),],
    "endtime": [np.datetime64("2020-01-01T00:00:00") + np.timedelta64(int(t2*1000), 'ms') - np.timedelta64(int(dt*1000), 'ms'), 
                np.datetime64("2020-01-01T00:00:00") + np.timedelta64(int(t2//2*1000), 'ms') - np.timedelta64(int(dt*1000), 'ms'),
                np.datetime64("2020-01-01T00:00:00") + np.timedelta64(int(t2//2*1000), "ms") - np.timedelta64(int(dt*1000), "ms")
               ],
    "ray_path_length_m": [2000, 2000, 2000],
    "hyp_distance_m": [2000, 2000, 2000],
}
df = pd.DataFrame(df_contents, columns=df_contents.keys())
df.set_index(['phase_hint', 'event_id', 'seed_id_less', 'seed_id'], inplace=True)
df["sampling_rate"] = 1/dt
sg = sg.new_from_dict(data=df)  # This is a weird way to do this

st = Stream()
tr = Trace(data=source_disp, header={
    "starttime": to_utc(sg.data.iloc[0].starttime),
    "delta": dt,
    "network": "UK",
    "station": "STA1",
    "channel": "HHZ",
})
st.append(tr)
tr = Trace(data=dis_source_disp, header={
    "starttime": to_utc(sg.data.iloc[1].starttime),
    "delta": dt,
    "network": "UK",
    "station": "STA2",
    "channel": "HHZ",
})
st.append(tr)
tr = Trace(data=zp_source_disp, header={
    "starttime": to_utc(sg.data.iloc[2].starttime),
    "delta": dt,
    "network": "UK",
    "station": "STA3",
    "channel": "HHZ",
})
st.append(tr)
tg = TraceGroup(sg, st.copy(), "displacement").fillna()  # START HERE... MAKE SURE TO CREATE THE STREAM BEFORE ADDING THE ZERO PADDING TO THE END TO ENSURE WE'RE GETTING THE RIGHT SAMPLE LENGTHS? BUT IT STILL SEEMS LIKE THERE WOULD BE A SAMPLE LENGTH DEPENDENCE THAT IS NOT ACCOUNTED FOR...

spec = tg.dft()

In [ ]:
len(sg.data)

In [ ]:
st.plot()

## Convert to the frequency domain and plot

In [ ]:
def get_fft(ar, dt):
    """Return the fft of the array and its """
    fft_ar = np.fft.rfft(ar)
    freq = np.fft.rfftfreq(len(ar), dt)
    return freq, fft_ar

In [ ]:
freqs, source_disp_fft = get_fft(source_disp, dt)
freqs, dis_source_disp_fft = get_fft(dis_source_disp, dt)
freqs, zp_source_disp_fft = get_fft(zp_source_disp, dt)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))
ax1.loglog(freqs, abs(source_disp_fft))
ax2.loglog(freqs, abs(dis_source_disp_fft))
ax3.loglog(freqs, abs(zp_source_disp_fft))
ax1.set_ylabel('amplitude')

ax1.loglog(spec.data.columns, abs(spec.data.iloc[0]), alpha=0.8)
ax2.loglog(spec.data.columns, abs(spec.data.iloc[1]), alpha=0.8)
ax3.loglog(spec.data.columns, abs(spec.data.iloc[2]), alpha=0.8)

### Convert back to the time domain and plot to make sure you get the input back

Note that it is necessary to divide by the sample spacing to get the correct amplitude...

In [ ]:
source_disp_rev = np.fft.irfft(source_disp_fft)
dis_source_disp_rev = np.fft.irfft(dis_source_disp_fft)
zp_source_disp_rev = np.fft.irfft(zp_source_disp_fft)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))
ax1.plot(x, source_disp_rev)
ax2.plot(x, dis_source_disp_rev)
ax3.plot(x, zp_source_disp_rev)
# plt.plot(x, source_disp)
ax1.set_ylabel('displacement amplitude (m)')

## Do the derivative in the time domain

In [ ]:
vel = np.gradient(source_disp, dt)
dis_vel = np.gradient(dis_source_disp, dt)
zp_vel = np.gradient(zp_source_disp, dt)

## Repeat in the frequency domain and convert back to the time domain

In [ ]:
vel_fft = 1j * 2 * np.pi * freqs * source_disp_fft
vel_td = np.fft.irfft(vel_fft)

dis_vel_fft = 1j * 2 * np.pi * freqs * dis_source_disp_fft
dis_vel_td = np.fft.irfft(dis_vel_fft)

zp_vel_fft = 1j * 2 * np.pi * freqs * zp_source_disp_fft
zp_vel_td = np.fft.irfft(zp_vel_fft)

In [ ]:
spec_v = spec.to_motion_type("velocity")
spec_to_td = np.fft.irfft(spec_v.data, axis=-1)

## Plot the analytical, numerical, and fft solutions and compare

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))

ax1.plot(x, source_vel)
ax1.plot(x, vel)
ax1.plot(x, vel_td, alpha=0.5)

ax2.plot(x, dis_source_vel)
ax2.plot(x, dis_vel)
ax2.plot(x, dis_vel_td, alpha=0.5)

ax3.plot(x, zp_source_vel)
ax3.plot(x, zp_vel)
ax3.plot(x, zp_vel_td, alpha=0.5)
ax1.set_ylabel('velocity amplitude (m/s)')

ax1.set_ylim(-0.05, 0.05)
ax2.set_ylim(-0.25, 0.25)

ax1.plot(x[:-100], spec_to_td[0], alpha=0.8)
ax2.plot(x[:-100], spec_to_td[1], alpha=0.8)
ax3.plot(x[:-100], spec_to_td[2], alpha=0.8)
# ax2.loglog(spec_v.data.columns, abs(spec_v.data.iloc[1]), alpha=0.8)
# ax3.loglog(spec_v.data.columns, abs(spec_v.data.iloc[2]), alpha=0.8)

In [ ]:
print("Max Diffs:", (source_vel - vel_td).max(), (dis_source_vel - dis_vel_td).max(), (zp_source_vel - zp_vel_td).max())
print("Max Diffs:", (source_vel[:-100] - spec_to_td[0]).max(), (dis_source_vel[:-100] - spec_to_td[1]).max(), (zp_source_vel[:-100] - spec_to_td).max())

### Plot the FFTs from each solution and compare

In [ ]:
source_vel_fft = np.fft.rfft(source_vel)
dis_source_vel_fft = np.fft.rfft(dis_source_vel)
zp_source_vel_fft = np.fft.rfft(zp_source_vel)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))

ax1.loglog(freqs, abs(source_vel_fft))
ax1.loglog(freqs, abs(np.fft.rfft(vel)))
ax1.loglog(freqs, abs(vel_fft), alpha=0.5)

ax2.loglog(freqs, abs(dis_source_vel_fft))
ax2.loglog(freqs, abs(np.fft.rfft(dis_vel)))
ax2.loglog(freqs, abs(dis_vel_fft), alpha=0.5)
           
ax3.loglog(freqs, abs(zp_source_vel_fft))
ax3.loglog(freqs, abs(np.fft.rfft(zp_vel)))
ax3.loglog(freqs, abs(zp_vel_fft), alpha=0.5)

ax1.loglog(spec_v.data.columns, abs(spec_v.data.iloc[0]))
ax2.loglog(spec_v.data.columns, abs(spec_v.data.iloc[1]))
ax3.loglog(spec_v.data.columns, abs(spec_v.data.iloc[2]))

In [ ]:
print(abs(source_vel_fft).mean() - abs(spec_v.data.iloc[0]).mean(), 
      abs(dis_source_vel_fft).mean() - abs(spec_v.data.iloc[1]).mean(), 
      abs(zp_source_vel_fft).mean() - abs(spec_v.data.iloc[2]).mean())

## Energy
Next we estimate energy and the equivalent in the freq domain.

## Time domain energy

In [ ]:
source_energy = source_vel ** 2
dis_source_energy = dis_source_vel ** 2
zp_source_energy = zp_source_vel ** 2
spec_energy = np.fft.irfft(spec_v.data, axis=1) ** 2

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))
ax1.plot(x, source_energy)
ax1.plot(x[:-100], spec_energy[0], alpha=0.5)
ax2.plot(x, dis_source_energy)
ax2.plot(x[:-100], spec_energy[1], alpha=0.5)
ax3.plot(x, zp_source_energy)
ax3.plot(x[:-100], spec_energy[2], alpha=0.5)
ax1.set_ylabel('amplitude (m^2/s^2)')

In [ ]:
# energy_td = np.trapz(source_energy, dx=dt)
energy_td = np.sum(source_energy)
dis_energy_td = np.sum(dis_source_energy[0:len(x)//2])  # This should be half the energy of the first (and zero padding shouldn't make a big difference?)
zp_energy_td = np.sum(zp_source_energy[0:len(x)//2])  # This should be twice the energy of the first because it's higher frequency (again, zero padding doesn't actuall matter)
print(energy_td, dis_energy_td, zp_energy_td)
print(np.sum(spec_energy, axis=1))

## Frequency domain energy

In [ ]:
def get_ppsd(fft, n, dt):
    """Get the power spectral density? Or something close to it ;)"""
#     print(abs(fft).min(), abs(fft).max())
    fft_ar_sq = fft ** 2
#     print(abs(fft_ar_sq).min(), abs(fft_ar_sq).max())
    out = fft_ar_sq * (dt / n)
#     print(abs(out).min(), abs(out).max())
    # double non zero components to account for neg. frequencies
    out[1:] *= 2
#     print(abs(out).min(), abs(out).max())
    freq = np.fft.rfftfreq(n, dt)
    return freq, out

In [ ]:
freqs, ppsd_analytic = get_ppsd(np.fft.rfft(source_vel), len(source_vel), dt) # Analytical
freqs, ppsd_time = get_ppsd(np.fft.rfft(vel), len(source_vel), dt) # Get velocity in time domain (should pretty closely match the analytical solution)
freqs, ppsd_freq = get_ppsd(vel_fft, len(source_vel), dt) # Get velocity in frequency domain

freqs, dis_ppsd_analytic = get_ppsd(np.fft.rfft(dis_source_vel), len(dis_source_vel), dt) # Analytical
freqs, dis_ppsd_time = get_ppsd(np.fft.rfft(dis_vel), len(dis_source_vel), dt) # Get velocity in time domain
freqs, dis_ppsd_freq = get_ppsd(dis_vel_fft, len(dis_source_vel), dt) # Get velocity in frequency domain

freqs, zp_ppsd_analytic = get_ppsd(np.fft.rfft(zp_source_vel), len(zp_source_vel), dt) # Analytical
freqs, zp_ppsd_time = get_ppsd(np.fft.rfft(zp_vel), len(zp_source_vel), dt) # Get velocity in time domain
freqs, zp_ppsd_freq = get_ppsd(zp_vel_fft, len(zp_source_vel), dt) # Get velocity in frequency domain

In [ ]:
spec_ppsd = spec_v.to_spectra_type("psd")

In [ ]:
spec_freqs, spec_ppsd1 = get_ppsd(spec_v.data.values, len(tg.data.columns), dt)

In [ ]:
spec_ppsd1.shape

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))

ax1.loglog(freqs, abs(ppsd_analytic))
ax1.loglog(freqs, abs(ppsd_time))
ax1.loglog(freqs, abs(ppsd_freq), alpha=0.5)

ax2.loglog(freqs, abs(dis_ppsd_analytic))
ax2.loglog(freqs, abs(dis_ppsd_time))
ax2.loglog(freqs, abs(dis_ppsd_freq), alpha=0.5)

ax3.loglog(freqs, abs(zp_ppsd_analytic))
ax3.loglog(freqs, abs(zp_ppsd_time))
ax3.loglog(freqs, abs(zp_ppsd_freq), alpha=0.5)

ax1.set_ylabel('amplitude (m^2/s^2)')

ax1.loglog(spec_ppsd.data.columns, abs(spec_ppsd.data.iloc[0]), alpha=0.8)
ax2.loglog(spec_ppsd.data.columns, abs(spec_ppsd.data.iloc[1]), alpha=0.8)
ax3.loglog(spec_ppsd.data.columns, abs(spec_ppsd.data.iloc[2]), alpha=0.8)

ax1.loglog(spec_ppsd.data.columns, abs(spec_ppsd1[0]), alpha=0.8)
ax2.loglog(spec_ppsd.data.columns, abs(spec_ppsd1[1]), alpha=0.8)
ax3.loglog(spec_ppsd.data.columns, abs(spec_ppsd1[2]), alpha=0.8)

In [ ]:
power_fd = np.sum(abs(spec_ppsd.data), axis=1) 
power_fd1 = np.sum(abs(spec_ppsd1), axis=1)

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=["Integrated Velocity", "Analytical Velocity by PPSD", "Time Domain Velocity by PPSD", "Frequency Domain Velocity by PPSD", "Ratio"])
# Continuous pulse
integrated = np.trapz(source_vel**2, dx=dt)
analytic = np.sum(abs(ppsd_analytic))
td = np.sum(abs(ppsd_time))
fd = np.sum(abs(ppsd_freq))
ratio = td/fd
df.loc["Continuous"] = [integrated, analytic, td, fd, ratio]
# Discontinuity
integrated = np.trapz(dis_source_vel**2, dx=dt)
analytic = np.sum(abs(dis_ppsd_analytic))
td = np.sum(abs(dis_ppsd_time))
fd = np.sum(abs(dis_ppsd_freq))
ratio = td/fd
df.loc["Discontinuity"] = [integrated, analytic, td, fd, ratio]
# Zero padded (half-width pulse)
integrated = np.trapz(zp_source_vel**2, dx=dt)
analytic = np.sum(abs(zp_ppsd_analytic))
td = np.sum(abs(zp_ppsd_time))
fd = np.sum(abs(zp_ppsd_freq))
ratio = td/fd
df.loc["Zero Padded"] = [integrated, analytic, td, fd, ratio]
df.loc["Continuous", "MoPy"] = power_fd[0]
df.loc["Discontinuity", "MoPy"] = power_fd[1]
df.loc["Zero Padded", "MoPy"] = power_fd[2]
df.loc["Continuous", "MoPy_hybrid"] = power_fd1[0]
df.loc["Discontinuity", "MoPy_hybrid"] = power_fd1[1]
df.loc["Zero Padded", "MoPy_hybrid"] = power_fd1[2]

In [ ]:
df

Note: the mopy hybrid results are off as a result of how columns get selected on dataframes when their values are numeric. I'm leaving that in here to illustrate a point, because it took me so long to figure out what it was doing.